In [33]:
import tensorflow as tf
import numpy as np 
import tensorflow.compat.v1 as tf

#si vous avez un tf 2.x
tf.disable_v2_behavior() 

import datetime
import pandas as pd
import re

In [80]:
df = pd.read_csv('preprocessed_data.csv')
df.columns
df

,Unnamed: 0,Id_Q,OwnerUserId_Q,CreationDate_Q,Score_Q,Title,Body_Q,Id_A,OwnerUserId_A,CreationDate_A,ParentId,Score_A,Body_A
0,0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",497,50.0,2008-08-02T16:56:53Z,469,4,"['p', 'open', 'terminal', 'application', 'gt',..."
1,1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",518,153.0,2008-08-02T17:42:28Z,469,2,"['p', 'able', 'find', 'anything', 'directly', ..."
2,2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",3040,457.0,2008-08-06T03:01:23Z,469,12,"['p', 'unfortunately', 'api', 'deprecated', 'l..."
3,3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",195170,745.0,2008-10-12T07:02:40Z,469,1,"['p', 'must', 'method', 'cocoa', 'get', 'list'..."
4,4,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,"['p', 'cross', 'platform', 'python', 'applicat...",536,161.0,2008-08-02T18:49:07Z,502,9,"['p', 'use', 'imagemagick', 'convert', 'utilit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788452,168352.0,2009-11-24T07:32:48Z,1788432,3,"['p', 'code', 'http', 'example', '20home', 'co..."
49996,49996,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788459,13992.0,2009-11-24T07:34:19Z,1788432,0,"['p', 'could', 'consider', 'adding', 'url', 'f..."
49997,49997,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788715,55562.0,2009-11-24T08:40:26Z,1788432,0,"['blockquote', 'p', 'handle', 'thing', 'p', 'b..."
49998,49998,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1790175,217754.0,2009-11-24T13:48:56Z,1788432,6,"['p', 'keep', 'url', 'pretty', 'example', 'use..."


In [89]:
ref_time = datetime.datetime(2008, 1, 1)

def to_integer(dt_timeQ, dt_timeA):
    timeQ = dt_timeQ - ref_time
    timeA = dt_timeA - ref_time
    time_diff = 1 - (timeQ.days*86400 + timeQ.seconds) / (timeA.days*86400 + timeA.seconds)
    return time_diff

    
def get_time_diff(df):
    for index, row in df.iterrows():
        row.CreationDate_A = datetime.datetime.strptime(row.CreationDate_A, "%Y-%m-%dT%H:%M:%SZ")
        row.CreationDate_Q = datetime.datetime.strptime(row.CreationDate_Q, "%Y-%m-%dT%H:%M:%SZ")
        df.at[index, 'time_diff'] = to_integer(row.CreationDate_Q, row.CreationDate_A)
    return df


df = get_time_diff(df)
df['score'] = df['Score_A'] + df['Score_A'] * df['time_diff']

In [92]:
df['score'] = df['Score_A'] + df['Score_A'] * df['time_diff']

,Unnamed: 0,Id_Q,OwnerUserId_Q,CreationDate_Q,Score_Q,Title,Body_Q,Id_A,OwnerUserId_A,CreationDate_A,ParentId,Score_A,Body_A,time_diff,score
0,0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",497,50.0,2008-08-02T16:56:53Z,469,4,"['p', 'open', 'terminal', 'application', 'gt',...",0.000342,4.001366
1,1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",518,153.0,2008-08-02T17:42:28Z,469,2,"['p', 'able', 'find', 'anything', 'directly', ...",0.000489,2.000978
2,2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",3040,457.0,2008-08-06T03:01:23Z,469,12,"['p', 'unfortunately', 'api', 'deprecated', 'l...",0.016014,12.192172
3,3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,"['p', 'using', 'photoshop', 'javascript', 'api...",195170,745.0,2008-10-12T07:02:40Z,469,1,"['p', 'must', 'method', 'cocoa', 'get', 'list'...",0.247677,1.247677
4,4,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,"['p', 'cross', 'platform', 'python', 'applicat...",536,161.0,2008-08-02T18:49:07Z,502,9,"['p', 'use', 'imagemagick', 'convert', 'utilit...",0.000346,9.003118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788452,168352.0,2009-11-24T07:32:48Z,1788432,3,"['p', 'code', 'http', 'example', '20home', 'co...",0.000007,3.000022
49996,49996,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788459,13992.0,2009-11-24T07:34:19Z,1788432,0,"['p', 'could', 'consider', 'adding', 'url', 'f...",0.000009,0.000000
49997,49997,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1788715,55562.0,2009-11-24T08:40:26Z,1788432,0,"['blockquote', 'p', 'handle', 'thing', 'p', 'b...",0.000075,0.000000
49998,49998,1788432,215993.0,2009-11-24T07:25:37Z,2,How to have a URL like this in Django?,"['p', 'url', 'like', 'code', 'example', 'categ...",1790175,217754.0,2009-11-24T13:48:56Z,1788432,6,"['p', 'keep', 'url', 'pretty', 'example', 'use...",0.000384,6.002303
